In [1]:
import numpy as np

In [2]:
'''
Convolution class using zero padding

param last_layer - point to last layer, which pass the value over
param input_num - numbers of input feature maps/images 
param input_size - input feature maps/images size
param filter_size - size of the filter/kernel
param filter_num - numbers of filters, refer to number of output feature maps/images 
param stride - moving step of the kernel function
'''
class ConvLayer():
    def __init__(self, func, d_func, last_layer, input_num, input_size, filter_size, filter_num, stride):
        # Activation Functions
        self.act_func = func                                      # Activation function
        self.d_act_func = d_func                                  # Derivative of activate function
        
        self.last_layer = last_layer
        self.input_num = input_num
        self.input_size = input_size
        self.filter_size = filter_size
        self.filter_num = filter_num
        self.stride = stride
        
        # Initial kernel
        self.kernel = np.random.randn(output_num,input_num,filter_size,filter_size) / np.sqrt(filter_size*filter_size)
        # Initial bias
        self.bias = np.random.randn(self.filter_num)
        
        # Check if the parameters of input size, filter size, stride is legel
        output_size = (input_size + stride - filter_size) / stride
        if(output_size%1 != 0):
            print("Invalid ! Please check your parameters");
            return -1
        
        self.input_img = 0
        self.stride_shape = (int(input_num),int(output_size),int(output_size),int(filter_size),int(filter_size))
        self.strides = (int(input_size*input_size*8), int(input_size*stride*8), int(stride*8), int(input_size*8), 8)
        self.conv_img = 0
        self.output_img = 0
        self.d_func_img = 0
        
        self.delta_bias = 0      # Correction of bias
        self.pass_error = 0      # Error passed to previous layer
        self.delta_w = 0         # Weight correction
        self.error = 0           # input error * derivative of activation function
        
    def forwrad_pass(self):
        strided_img = np.lib.stride_tricks.as_strided(self.input_img, shape=self.stride_shape, strides=self.strides)       # Cut the image with kernel size
        
        # Convolution operations
        self.output_img = np.einsum("ijklm,ailm->ajk", strided_img, self.kernel)
        for i in range(self.filter_num):
            self.conv_img[i] = self.output_img[i] + self.bias[i]     # Add bias
            
        self.output_img = self.act_func(self.conv_img)               # Through activation function
        self.d_func_img = self.d_act_func(self.conv_img)             # Through derivative of activation function
    
    '''
    Adjust weights, using backpropagation
    For error function, e = y_predict - y_desire
    For weight correction, w_n+1 = w_n - delta_w
    '''
    def adjust_weight(self, lr_rate):
        self.pass_error = np.zeros((input_num, input_size, input_size), dtype=np.float)                 # delta_b
        self.delta_w = np.zeros((output_num,input_num,filter_size,filter_size), dtype=np.float)         
        self.error = self.d_func_img * self.bp_vec[feature][img_h][img_w]                              # error
        for img_h in range(len(self.passed_error[0])):
            for img_w in range(len(self.passed_error[0][0])):
                left_corner_h = img_h * stride
                left_corner_w = img_w * stride
                for feature in range(len(self.passed_error)):
                    # error pass to previous layer
                    self.pass_error[:,left_corner_h:(left_corner_h+filter_size),left_corner_w:(left_corner_w+filter_size)] += kernel[feature] * self.error[feature][img_h][img_w]
                    # weight adjustment value
                    self.delta_w[feature][:] += self.input_img[:,left_corner_h:(left_corner_h+filter_size),left_corner_w:(left_corner_w+filter_size)] * self.error[feature][img_h][img_w]
        
        self.delta_bias = self.passed_error.copy()
    
    def extract_value(self):
        self.input_img = self.last_layer.get_output()
        return self.input_img
    
    def get_output(self):
        return self.output_img.copy()
    
    '''
    Pass backpropagation value back to previous layer
    '''
    def pass_bp(self, bp_value):
        self.bp_vec = bp_value.copy()